In [1]:
# 该代码为获取上海瑞幸POI数据
# 第一步：获取上海边界范围外接矩形的坐标值
# 第二步：以四叉树索引的方式不断四分矩形，直到所有矩形符合POI数量限制
# 第三步：用来存放不符合POI数量限制多边形的列表
# 第四步：保存POI数据

import geopandas as gp #### geopandas的安装是个挑战, 上网仔细查询
import pandas as pd
import requests

requests.packages.urllib3.disable_warnings()


# 判断输入多边形是否满足POI数量限制
# 假设POI数量限制为100
# 单条URL查询20条POI
# 则需要判断第6页的POI数量是否0
# 如果是0，则当前多边形满足POI数量限制，返回True
# 如果不为0，则当前多边形不满足POI数量限制，返回False

In [2]:
# 变量
key = "2f7cc3cf5f5daba8a4306a3b8fa0826f"  # 高德地图API密钥，需要在高德官网申请，操作手册：https://kdocs.cn/l/coOAaCYwW0tg
keywords = '肯德基'
pathShanghai = "beijing.geojson"  # 上海边界
savePoiList = []  # 存放POI数据的结果列表
savePath = "bjxcpoiresult.csv"  # POI数据存放地址

In [3]:
# 判断输入多边形是否满足POI数量限制
# 假设POI数量限制为100
# 单条URL查询20条POI
# 则需要判断第6页的POI数量是否0
# 如果是0，则当前多边形满足POI数量限制，返回True
# 如果不为0，则当前多边形不满足POI数量限制，返回False
def judgeIfSatisfy_GaoDe(inputPolygon):
    # 组成Url
    currentUrl = "https://restapi.amap.com/v3/place/polygon?polygon=" + str(inputPolygon[0]) + "," + str(
        inputPolygon[2]) + "|" + str(inputPolygon[1]) + "," + str(
        inputPolygon[2]) + "|" + str(inputPolygon[1]) + "," + str(inputPolygon[3]) + "|" + str(
        inputPolygon[0]) + "," + str(inputPolygon[3]) + "|" + str(inputPolygon[0]) + "," + str(
        inputPolygon[2]) + "&offset=20&page=6&keywords="+ keywords + "&types=050000&output=json&key=" + key
    # 发送Get请求，并接收返回内容
    response = requests.get(currentUrl, stream=True, verify=False, timeout=60)
    returnData = response.json()
    # 获取数量POI数量
    resultCount = len(returnData['pois'])
    response.close()
    # 判断是否为0，并输出结果
    if resultCount > 0:
        return False
    else:
        return True

In [4]:
# 执行四叉树
# 对输入矩形进行四分
# 等分为左下角、右下角、右上角、左上角4个矩形
def executeQuadtree(minLon, maxLon, minLat, maxLat):
    resultList = []
    dLon = (maxLon - minLon) / 2
    dLat = (maxLat - minLat) / 2
    # 左下角的矩形
    resultList.append([minLon, minLon + dLon, minLat, minLat + dLat])
    # 右下角的矩形
    resultList.append([minLon + dLon, minLon + 2 * dLon, minLat, minLat + dLat])
    # 右上角的矩形
    resultList.append([minLon + dLon, minLon + 2 * dLon, minLat + dLat, minLat + 2 * dLat])
    # 左上角的矩形
    resultList.append([minLon, minLon + dLon, minLat + dLat, minLat + 2 * dLat])
    return resultList

In [5]:
# 从单个矩形获取瑞幸POI的方法
def getPoiFromPolygon(inputPolygon):
    # 存放POI数据结果的列表
    resultList = []
    # 由于POI数量限制是100，每页返回POI数量是20
    # 因此，最大的页数为5
    for currentPage in range(1, 6):
        # 组成Url
        currentUrl = "https://restapi.amap.com/v3/place/polygon?polygon=" + str(inputPolygon[0]) + "," + str(
            inputPolygon[2]) + "|" + str(inputPolygon[1]) + "," + str(
            inputPolygon[2]) + "|" + str(inputPolygon[1]) + "," + str(inputPolygon[3]) + "|" + str(
            inputPolygon[0]) + "," + str(inputPolygon[3]) + "|" + str(inputPolygon[0]) + "," + str(
            inputPolygon[2]) + "&offset=20&page=" + str(
            currentPage) + "&keywords="+ keywords +"&types=050000&output=json&key=" + key
        # 发送Get请求，并接收返回内容
        response = requests.get(currentUrl, stream=True, verify=False, timeout=60)
        returnData = response.json()
        returnPoiList = returnData['pois']
        if len(returnPoiList) > 0:
            for i in range(0, len(returnPoiList)):
                saveName = returnPoiList[i]['name']
                saveType = returnPoiList[i]['type']
                saveAddress = returnPoiList[i]['address']
                saveLocation = returnPoiList[i]['location']
                saveProvince = returnPoiList[i]['pname']
                saveCity = returnPoiList[i]['cityname']
                saveArea = returnPoiList[i]['adname']
                resultList.append(
                    [saveName, saveType, saveAddress, saveLocation, saveProvince, saveCity, saveArea])
                print(saveName, saveType, saveAddress, saveLocation, saveProvince, saveCity, saveArea)
        else:
            # 如果当前页POI数量为0，则返回已获取的POI数据并跳出
            return resultList
    return resultList

In [6]:
print("=======================================")
print("第一步：获取边界范围外接矩形的坐标值")
print("=======================================")
# 第一步：获取上海边界范围外接矩形的坐标值
# 通过读取GeoJSON格式的上海边界数据
# 获取上海边界外接矩形的最小经度、最大经度、最小纬度、最大纬度

# 上海边界经纬度值
resultMinLon = 99999.0  # 最小经度
resultMaxLon = 0.0  # 最大经度
resultMinLat = 99999.0  # 最小纬度
resultMaxLat = 0.0  # 最大纬度

# 读取上海边界数据
loadGeoData = gp.read_file(pathShanghai)
for i in range(0, len(loadGeoData)):
    # 读取几何数据
    loadGeometry = loadGeoData.loc[i, "geometry"]
    # 上海边界的几何数据是由多个多边形组成，如崇明岛、长兴岛、上海市区等
    # 所以需要遍历所有多边形
    # 多边形的类型是Polygon或者MultiPolygon
    for j in range(0, len(loadGeometry)):
        loadPolygon = loadGeometry[j]
        # 读取多边形的边界
        # 边界的类型是LineString(单线)或MultiLineString(多线)
        if loadPolygon.boundary.geom_type == 'LineString':
            # 读取边界中的每个坐标点
            for z in range(0, len(loadPolygon.boundary.coords)):
                loadLon = loadPolygon.boundary.coords[z][0]  # 经度
                loadLat = loadPolygon.boundary.coords[z][1]  # 纬度
                # 进行大小判断，并替换结果
                if loadLon <= resultMinLon:
                    resultMinLon = loadLon
                if loadLon >= resultMaxLon:
                    resultMaxLon = loadLon
                if loadLat <= resultMinLat:
                    resultMinLat = loadLat
                if loadLat >= resultMaxLat:
                    resultMaxLat = loadLat
        elif loadPolygon.boundary.geom_type == 'MultiLineString':
            # 如果边界是MultiLineString，则需要遍历其中的每条LineString
            for z in range(0, len(loadPolygon.boundary[0].coords)):
                loadLon = loadPolygon.boundary[0].coords[z][0]
                loadLat = loadPolygon.boundary[0].coords[z][1]
                if loadLon <= resultMinLon:
                    resultMinLon = loadLon
                if loadLon >= resultMaxLon:
                    resultMaxLon = loadLon
                if loadLat <= resultMinLat:
                    resultMinLat = loadLat
                if loadLat >= resultMaxLat:
                    resultMaxLat = loadLat
# 结果输出
print("最小经度：", resultMinLon)
print("最大经度：", resultMaxLon)
print("最小纬度：", resultMinLat)
print("最大纬度：", resultMaxLat)

第一步：获取边界范围外接矩形的坐标值
最小经度： 115.423411
最大经度： 117.514583
最小纬度： 39.442758
最大纬度： 41.0608


C:\Users\quzhe\AppData\Local\Temp\ipykernel_35240\2503134791.py:22: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for j in range(0, len(loadGeometry)):
C:\Users\quzhe\AppData\Local\Temp\ipykernel_35240\2503134791.py:23: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  loadPolygon = loadGeometry[j]


In [7]:
print("=======================================")
print("第二步：以四叉树索引的方式不断四分矩形，直到所有矩形符合POI数量限制")
print("=======================================")
resultPolygonList = []  # 存放符合POI数量限制多边形的列表
currentPolygonList = [[resultMinLon, resultMaxLon, resultMinLat, resultMaxLat]]  # 用来存放不符合POI数量限制多边形的列表

# 以四叉树的方式不断四分外接矩形
for currentLevel in range(1, 9999999):
    # 用来存放不符合POI数量限制多边形的临时列表
    tempPolygonList = []
    for z in range(0, len(currentPolygonList)):
        loadMinLon = currentPolygonList[z][0]
        loadMaxLon = currentPolygonList[z][1]
        loadMinLat = currentPolygonList[z][2]
        loadMaxLat = currentPolygonList[z][3]
        # 判断当前矩形是否符合POI数量限制
        # 如果符合则放入resultPolygonList
        # 如果不符合则进行四分，并将四分结果放入tempPolygonList
        ifSatisfy = judgeIfSatisfy_GaoDe([loadMinLon, loadMaxLon, loadMinLat, loadMaxLat])
        if ifSatisfy == True:
            resultPolygonList.append([loadMinLon, loadMaxLon, loadMinLat, loadMaxLat])
        else:
            # 进行四叉树
            tempQuadtree = executeQuadtree(loadMinLon, loadMaxLon, loadMinLat, loadMaxLat)
            # 左下角
            tempPolygonList.append(tempQuadtree[0])
            # 右下角
            tempPolygonList.append(tempQuadtree[1])
            # 右上角
            tempPolygonList.append(tempQuadtree[2])
            # 左上角
            tempPolygonList.append(tempQuadtree[3])
    currentPolygonList = tempPolygonList.copy()
    # 如果没有多边形进入下一层级，则跳出
    print("第", currentLevel, "层", "......", "符合要求的矩形：", len(resultPolygonList), "......", "剩余矩形：",
          len(currentPolygonList))
    if len(currentPolygonList) == 0:
        break

第二步：以四叉树索引的方式不断四分矩形，直到所有矩形符合POI数量限制
第 1 层 ...... 符合要求的矩形： 1 ...... 剩余矩形： 0


In [8]:
print("=======================================")
print("第三步：遍历所有矩形，获取北京星巴克POI")
print("=======================================")
for i in range(0, len(resultPolygonList)):
    # 读取每个矩形
    loadPolygon = resultPolygonList[i]
    # 获取矩形中的POI数据，并添加到结果列表中
    savePoiList.extend(getPoiFromPolygon(loadPolygon))

第三步：遍历所有矩形，获取北京星巴克POI
喜茶(北京金融街购物中心店) 餐饮服务;冷饮店;冷饮店 金城坊街9号楼一层F2-002号商铺 116.361675,39.917218 北京市 北京市 西城区
喜茶(北京君太百货DP店) 餐饮服务;冷饮店;冷饮店 西单北大街130号君太百货2楼f-08 116.373360,39.909208 北京市 北京市 西城区
喜茶(北京前门大街店) 餐饮服务;冷饮店;冷饮店 前门大街A2-2号(前门地铁站I西南口步行80米) 116.398255,39.898173 北京市 北京市 东城区
喜茶(北京凯德MALL西直门店) 餐饮服务;冷饮店;冷饮店 西直门外大街1号西环广场凯德Mall西直门1层49号 116.352384,39.942165 北京市 北京市 西城区
喜茶(北京apm店) 餐饮服务;冷饮店;冷饮店 王府井大街138号北京apm B1层B103A号商铺 116.412326,39.915196 北京市 北京市 东城区
喜茶(北京广安门天虹店) 餐饮服务;冷饮店;冷饮店 广安门外大街168号1幢1层101内043号商铺 116.341772,39.888651 北京市 北京市 西城区
喜茶(枫蓝国际购物中心店) 餐饮服务;冷饮店;冷饮店 西直门北大街32号院3号楼一层102-5至6号房间 116.356800,39.953438 北京市 北京市 海淀区
喜茶(北京崇文门国瑞店) 餐饮服务;冷饮店;冷饮店 崇文门外大街18号国瑞购物中心1幢F1-27号商铺 116.419294,39.898989 北京市 北京市 东城区
喜茶(北京翠微百货GO店) 餐饮服务;冷饮店;冷饮店 复兴路翠微百货B-F1-007号商铺 116.305911,39.907997 北京市 北京市 海淀区
喜茶(北京翠微印象城店) 餐饮服务;冷饮店;冷饮店 翠微街12号北京印象城一层45号 116.303391,39.912595 北京市 北京市 海淀区
喜茶(北京朝阳门外店) 餐饮服务;冷饮店;冷饮店 朝外大街路26号一层F1-08室商铺 116.441218,39.921743 北京市 北京市 朝阳区
喜茶(北京丽泽龙湖天街店) 餐饮服务;冷饮店;冷饮店 丽泽路16号院1号楼龙湖·北京丽泽天街商场(A-1F-16b,A-1F-17)号房屋 116

In [9]:
print("=======================================")
print("第四步：保存获取的POI数据")
print("=======================================")
pd.DataFrame(savePoiList, columns=["name", "type", "address", "location", "pname", "cityname", "adname"]).to_csv(
    savePath, index=False)
print("完成POI数据获取", "......", "总共", len(savePoiList), "个POI数据")

第四步：保存获取的POI数据
完成POI数据获取 ...... 总共 53 个POI数据
